In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from src import GetUserClimbData as gud
from src import AssesItemBased as assess
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
uid_path = './user_ids/uids.csv'
user_data = gud.get_users_climbs(uid_path)
user_data_idx = user_data.set_index('user_id')
# route_data = star_data.T  
# route_data = route_data.drop(labels='user_id', axis=0)
# cs_sim = cosine_similarity(route_data)

In [4]:
route_data = user_data.T  
route_data = route_data.drop(labels='user_id', axis=0)
cs_sim = cosine_similarity(route_data)


In [77]:
test_split = user_data_idx[:6]
train_split = user_data_idx[6:]
test_split.shape, train_split.shape

((6, 3590), (24, 3590))

In [83]:
held_out_routes, test_trim = assess.trim_routes(test_split,0.2)
train_held_out = train_split.append(test_trim)
train_held_out.shape

(30, 3590)

In [46]:
recs_fold_1 = assess.get_predictions(
    train_held_out,
    test_split,
    train_held_out.T,
    cs_sim)

In [68]:
judge, base = assess.format_for_galago(recs_fold_1,held_out_routes)
assess.export_data(judge,base,1)

In [6]:
assess.test_folds(user_data_idx,cs_sim,5)

In [28]:
metrics = assess.run_eval(5)

In [29]:
metrics

[[('MAP', '0.58210'), ('NDCG10', '0.99518')],
 [('MAP', '0.71366'), ('NDCG10', '0.99041')],
 [('MAP', '0.50101'), ('NDCG10', '0.98924')],
 [('MAP', '0.47236'), ('NDCG10', '0.98504')],
 [('MAP', '0.56327'), ('NDCG10', '0.98641')]]

In [37]:
names = np.array(metrics).T[0]
vals = np.array(metrics).T[1]

In [42]:
for i,metric in enumerate(names):
    print(metric[0],'on',folds,'folds')
    print('mean ',np.mean(vals[i].astype(float)))
    print('std ',np.mean(vals[i].astype(float)))

MAP
[0.5821  0.71366 0.50101 0.47236 0.56327]
NDCG10
[0.99518 0.99041 0.98924 0.98504 0.98641]
